In [27]:
import pandas
import numpy
import re
#Análisis de sentimientos
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json

In [45]:
def separaTopico(topico, df): #Separa todos los encabezados de acuerdo a su tópico
    listTopic = []
    dataArr = numpy.asarray(df)
    for i in range(len(dataArr)):
        if(dataArr[i][1] == topico):
            listTopic.append(dataArr[i][0])
    return listTopic

In [46]:
def limpiaTexto(text):
    "List all the word tokens in a text."
    return re.findall('[a-zA-Z0-9\-áéíóúÁÉÍÓÚ]{3,254}', text.lower())

In [47]:
def cleanTildes(palabra):
    s = palabra
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    
    return s

In [48]:
def limpiaPalabra(arrPalabras):
    raiz = []
    for i in range(len(arrPalabras)):
        #raiz.append(RaizSingular(cleanTildes(arrPalabras[i])))
        raiz.append(cleanTildes(arrPalabras[i]))
    
    return numpy.asarray(raiz, dtype=str)

In [49]:
def PalabrasTopico(listTopico): #Devuelve todas las palabras que aparecen en los titulares de las noticias 
    listHeadLines = []
    for i in range(len(listTopico)):
        #aux = nltk.word_tokenize(arrayTopico[i])
        aux = limpiaTexto(listTopico[i])
        #aux = limpiaPalabra()
        for j in range(len(aux)):
            aux2 = aux[j]
            listHeadLines.append(aux2)
            
    return numpy.asarray(listHeadLines)

In [33]:
def Probabilidad(palabrasTopico, palabrasUnicasTopico): #Devuelve un array con la ocurrencias de cada única del tópico
    Prob = numpy.zeros(len(palabrasUnicasTopico))
    for i in range(len(palabrasUnicasTopico)):
        Prob[i] = numpy.sum(palabrasTopico == palabrasUnicasTopico[i])
        Prob[i] = Prob[i] /len(palabrasTopico)
        
    return Prob

In [34]:
def Clasificador(arrayProbabilidades):
    indice = numpy.where(arrayProbabilidades == max(arrayProbabilidades))[0][0]
    #print("Max = ", max(arrayProbabilidades))
    if (indice == 0):
        return 'covid'
    if (indice == 1):
        return 'tecnologia'
    if (indice == 2):
        return 'deportes'
    if (indice == 3):
        return 'economia'
    if (indice == 4):
        return 'cultura'
    return 'otros'

In [50]:
def MAP(tweet, palabrasCovid, AlfabetoCovid, palabrasTecn, AlfabetoTecn, palabrasDep, AlfabetoDep, palabrasEcon, AlfabetoEcon, palabrasCult, AlfabetoCult):
    T = nltk.word_tokenize(tweet)
    Proba = numpy.ones(5)

    #Covid
    for j in range(len(T)):
        aux = numpy.sum(palabrasCovid == T[j]) + 1
        Proba[0] *= (aux + 1) / (len(AlfabetoCovid) + aux)
    
    #Tecnologia
    for j in range(len(T)):
        aux = numpy.sum(palabrasTecn == T[j]) 
        Proba[1] *= (aux + 1) / (len(AlfabetoTecn) + aux)

    #Deportes
    for j in range(len(T)):
        aux = numpy.sum(palabrasDep == T[j]) + 1
        Proba[2] *= (aux + 1) / (len(AlfabetoDep) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasEcon == T[j]) + 1
        Proba[3] *= (aux + 1) / (len(AlfabetoEcon) + aux)
        
    #Economía
    for j in range(len(T)):
        aux = numpy.sum(palabrasCult == T[j]) + 1
        Proba[4] *= (aux + 1) / (len(AlfabetoCult) + aux)

    for p in range(len(Proba)):
        Proba[p] = Proba[p] * 1/5
        
    return Clasificador(Proba)
        

In [51]:
def preProcesamiento(listaTitulares):
    titulares = []
    for titular in listaTitulares:
        aux = limpiaTexto(titular)
        titulares.append(str.join(' ', aux))
    
    return titulares


In [52]:
nombres = ['Titular', 'Tópico']
dataTrain = pandas.read_csv('.//TitularesTrain//titulares.csv', names=nombres)
dataTrain = dataTrain.dropna()
dataTrain

,Titular,Tópico
0,Cierran rastro de Mérida tras detectar 17 cont...,covid
2,El “peor escenario” para Tabasco eran 200 muer...,covid
4,Jalisco instalará espacios para enfermos no gr...,covid
6,Guerrero no está listo para iniciar la “nueva ...,covid
8,Reportan la muerte de 364 personas por covid-1...,covid
...,...,...
7846,"“El Cascanueces” de Disney: Cuento de hadas, m...",cultura
7848,"Rock, blues y música alternativa en Sinaloa",cultura
7850,María Mercedes Coroy es “Malinche” en Canal Once,cultura
7852,“Motivos para el canto y la danza. Poesía del 68”,cultura


In [53]:
palabrasCovid = limpiaPalabra(PalabrasTopico(separaTopico('covid', dataTrain)))
AlfabetoCovid = numpy.unique(palabrasCovid)

palabrasTecn = limpiaPalabra(PalabrasTopico(separaTopico('tecnologia', dataTrain)))
AlfabetoTecn = numpy.unique(palabrasTecn)

palabrasDep = limpiaPalabra(PalabrasTopico(separaTopico('deportes', dataTrain)))
AlfabetoDep = numpy.unique(palabrasDep)

palabrasEcon = limpiaPalabra(PalabrasTopico(separaTopico('economia', dataTrain)))
AlfabetoEcon = numpy.unique(palabrasEcon)

palabrasCult = limpiaPalabra(PalabrasTopico(separaTopico('cultura', dataTrain)))
AlfabetoCult = numpy.unique(palabrasCult)

Alfabeto = numpy.concatenate((AlfabetoCovid, AlfabetoTecn, AlfabetoDep, AlfabetoEcon, AlfabetoCult))
#Alfabeto.shape

In [54]:
ProbaCov = Probabilidad(palabrasCovid, AlfabetoCovid)
ProbaTecn = Probabilidad(palabrasTecn, AlfabetoTecn)
ProbaDep = Probabilidad(palabrasDep, AlfabetoDep)
ProbaEcon = Probabilidad(palabrasEcon, AlfabetoEcon)
ProbaCult = Probabilidad(palabrasCult, AlfabetoCult)

## Análisis de sentimientos

In [40]:
lexicon = pandas.read_csv('SEL/SEL_UTF8.txt', sep='\t', names=['Palabra', 'PFA', 'Categoría'])
lexicon.drop(['Categoría'], axis='columns', inplace=True)
lexicon.info()

Palabras = numpy.array(lexicon.Palabra)
PFA = numpy.array(lexicon.PFA)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2037 entries, 0 to 2036
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Palabra  2037 non-null   object
 1   PFA      2037 non-null   object
dtypes: object(2)
memory usage: 32.0+ KB


In [41]:
lista = []
for i in range(1, len(Palabras)):
    lista.append([Palabras[i], float(PFA[i])])
    
lexi = {a : b for a,b in lista}
lexi

{'abundancia': 0.83,
 'acabalar': 0.396,
 'acallar': 0.198,
 'acatar': 0.198,
 'acción': 0.397,
 'aceptable': 0.594,
 'aceptación': 0.696,
 'acicate': 0.429,
 'aclamación': 0.799,
 'aclamar': 0.799,
 'acogedor': 0.83,
 'acoger': 0.729,
 'acomodadamente': 0.729,
 'acuciar': 0.264,
 'acucioso': 0.232,
 'adecuar': 0.331,
 'adicto': 0.429,
 'admirable': 0.73,
 'admirablemente': 0.663,
 'admiración': 0.764,
 'admirar': 0.73,
 'admitir': 0.53,
 'adorable': 0.898,
 'adorablemente': 0.865,
 'adoración': 0.765,
 'adorador': 0.664,
 'adorar': 0.764,
 'afable': 0.696,
 'afán': 0.764,
 'afectivo': 0.864,
 'afecto': 0.899,
 'afectuosamente': 0.966,
 'afectuosidad': 0.932,
 'afectuoso': 0.898,
 'afervorizar': 0.731,
 'afición': 0.729,
 'aficionar': 0.595,
 'afortunadamente': 0.831,
 'afortunado': 0.932,
 'agradable': 0.899,
 'agradar': 0.798,
 'agradecer': 0.763,
 'agradecido': 0.764,
 'agudeza': 0.331,
 'aguzar': 0.199,
 'ahínco': 0.73,
 'ahíto': 0.564,
 'airoso': 0.565,
 'alabar': 0.53,
 'alboroza

In [42]:
SIA = SentimentIntensityAnalyzer()
SIA.lexicon.update(lexi)

In [43]:
SIA.lexicon.update(lexi)

In [44]:
print(json.dumps(SIA.lexicon, indent=2))

{
  "$:": -1.5,
  "%)": -0.4,
  "%-)": -1.5,
  "&-:": -0.4,
  "&:": -0.7,
  "( '}{' )": 1.6,
  "(%": -0.9,
  "('-:": 2.2,
  "(':": 2.3,
  "((-:": 2.1,
  "(*": 1.1,
  "(-%": -0.7,
  "(-*": 1.3,
  "(-:": 1.6,
  "(-:0": 2.8,
  "(-:<": -0.4,
  "(-:o": 1.5,
  "(-:O": 1.5,
  "(-:{": -0.1,
  "(-:|>*": 1.9,
  "(-;": 1.3,
  "(-;|": 2.1,
  "(8": 2.6,
  "(:": 2.2,
  "(:0": 2.4,
  "(:<": -0.2,
  "(:o": 2.5,
  "(:O": 2.5,
  "(;": 1.1,
  "(;<": 0.3,
  "(=": 2.2,
  "(?:": 2.1,
  "(^:": 1.5,
  "(^;": 1.5,
  "(^;0": 2.0,
  "(^;o": 1.9,
  "(o:": 1.6,
  ")':": -2.0,
  ")-':": -2.1,
  ")-:": -2.1,
  ")-:<": -2.2,
  ")-:{": -2.1,
  "):": -1.8,
  "):<": -1.9,
  "):{": -2.3,
  ");<": -2.6,
  "*)": 0.6,
  "*-)": 0.3,
  "*-:": 2.1,
  "*-;": 2.4,
  "*:": 1.9,
  "*<|:-)": 1.6,
  "*\\0/*": 2.3,
  "*^:": 1.6,
  ",-:": 1.2,
  "---'-;-{@": 2.3,
  "--<--<@": 2.2,
  ".-:": -1.2,
  "..###-:": -1.7,
  "..###:": -1.9,
  "/-:": -1.3,
  "/:": -1.3,
  "/:<": -1.4,
  "/=": -0.9,
  "/^:": -1.0,
  "/o:": -1.4,
  "0-8": 0.1,
  